In [3]:
import plotly.offline as off
import plotly.graph_objs as go
import numpy as np 
import pandas as pd


In [4]:
# читаем
d1 = np.load('monitoring_27.03_0-20.npy')
d2 = np.load('monitoring_27.03_20-40.npy')
d3 = np.load('monitoring_27.03_41-60.npy')
d4 = np.load('monitoring_27.03_61-80.npy')
d5 = np.load('monitoring_27.03_81-100.npy')

In [5]:
# соединяем
l = []
for i in [d1, d2, d3,d4,d5]:
    df = pd.DataFrame(i)
    l.extend(list(df.T.to_dict().values()))
   # print(pd.crosstab(columns=df['score'], index='count'))

df = pd.DataFrame(l)

df.columns = ['journal', 'title', 'abstract', 'doi', 'altmetrics', 'topic', 'chemscore', 'score']

In [6]:
# сводная статистика
pd.crosstab(columns=df['score'], index='count')

score     0  22.044091030955315  480.11848456412554  \
row_0                                                 
count  3122                   1                   1   

score  too short for similarities  
row_0                              
count                        1244

In [7]:
df = df.sort_values(by=['chemscore'])

In [19]:
# trace = go.Scatter(
#     x = df['new_chemscore'],
#     y = df['altmetrics'],
#     mode = 'markers'
# )

# data = [trace]

# # Plot and embed in ipython notebook!
# off.offline.plot(data)

'file://C:\\Users\\chuvakin\\Desktop\\ScientificSerach\\temp-plot.html'

In [8]:
import re

In [9]:
df['new_chemscore'] = [float(re.findall(r'\d\.?\d*',i)[0]) for i in df['chemscore']]

In [10]:
df['id'] = range(df.shape[0])

In [11]:
df['new_altmetrics'] = [*map(float, df['altmetrics'])]

In [34]:
df = df.sort_values(by=['new_chemscore','new_altmetrics'])

----

In [2]:
from bokeh.plotting import figure, show, output_file, ColumnDataSource, save
from bokeh.io import output_notebook, export_png
from bokeh.embed import file_html
from bokeh.resources import CDN

In [39]:

p1 = len(df['new_chemscore']) / 100

In [81]:
df['new_chemscore'][round(p1*85)]

1.0106814503669739

In [82]:
df['new_altmetrics'][round(p1*85)]

390.0

In [90]:
df['new_chemscore'].values

array([0., 0., 0., ..., 4., 4., 5.])

In [83]:
l = []
for i in df['new_chemscore'].values:
    if i < df['new_chemscore'][round(p1*85)]:
        l.append(0) 
    else: 
        l.append(1)
    
ll = []
for i in df['new_altmetrics'].values:
    if i < df['new_altmetrics'][round(p1*85)]:
        ll.append(0) 
    else: 
        ll.append(1)
lll = []        
for i, ii in zip(l, ll):
    lll.append(i + ii)    

In [84]:
lll.index(1)

723

In [85]:
df['colours'] = lll

In [86]:
colormap =  {0: "#00ffe0", 1: '#ff0000', 2: "#ffff00"}
df['hexes'] = df["colours"].apply(lambda c: colormap[c])

In [91]:
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
TOOLS2="box_zoom,reset,save"


source = ColumnDataSource(data=dict(
    x=df['new_chemscore'],
    y=df['new_altmetrics'],
    doi=df['doi'],
    he = df['hexes'],
    ids = list(df.index)
))
TOOLTIPS = [

    ("(chemscore,altmetrics)", "($x, $y)"),
    ("doi", "@doi"),
    ('id', "@ids"),
]
p = figure(plot_width=1400, plot_height=800, tooltips=TOOLTIPS,
           title="Science_articles - chemscore \ altmetrics", tools=TOOLS2)

p.circle('x', 'y', size=10, source=source, alpha=0.7, fill_color ='he')

output_notebook()

#r = p.circle('x', 'y', size=10, source=source, alpha=0.5, selection_color='green',nonselection_alpha=0.1, selection_alpha=0.5)

Loading BokehJS ...

In [66]:
show(p)

Loading BokehJS ...

In [59]:
def display_event(div, attributes=[], style = 'float:left;clear:left;font_size=10pt'):
    "Build a suitable CustomJS to display the current event in the div model."
    return CustomJS(args=dict(div=div), code="""
        var attrs = %s; var args = [];
        for (var i = 0; i<attrs.length; i++) {
            args.push(attrs[i] + '=' + Number(source[attrs[i]]).toFixed(2));
        }
        var line = "<span style=%r><b>" + cb_obj.event_name + "</b>(" + args.join(", ") + ")</span>\\n";
        var text = div.text.concat(line);
        var lines = text.split("\\n")
        if (lines.length > 35)
            lines.shift();
        div.text = lines.join("\\n");
    """ % (attributes, style))


# s1.selected.js_on_change('indices', CustomJS(args = dict(s1 = s1, s2 = s2), code = """
#         var inds = cb_obj.indices;
#         var d1 = s1.data;
#         d2 = {'x': [], 'y': []}
#         for (var i = 0; i < inds.length; i++) {
#             d2['x'].push(d1['x'][inds[i]])
#             d2['y'].push(d1['y'][inds[i]])
#         }
#         s2.data = d2  """))


div = Div(width=400, height=p.plot_height, height_policy="fixed")
button = Button(label="Button", button_type="success")
layout = column(button, row(p, div))

## Events with no attributes
button.js_on_event(events.ButtonClick, display_event(div)) # Button click
p.js_on_event(events.LODStart, display_event(div))         # Start of LOD display
p.js_on_event(events.LODEnd, display_event(div))           # End of LOD display

## Events with attributes
point_attributes = ['doi']
#point_attributes = ['x', 'y', 'sx', 'sy']                  # Point events
wheel_attributes = point_attributes + ['delta']            # Mouse wheel event
pan_attributes = point_attributes + ['delta_x', 'delta_y'] # Pan event
pinch_attributes = point_attributes + ['scale']            # Pinch event

point_events = [
    events.Tap, events.DoubleTap, events.Press,
    events.MouseMove, events.MouseEnter, events.MouseLeave,
    events.PanStart, events.PanEnd, events.PinchStart, events.PinchEnd,
]

for event in point_events:
    p.js_on_event(event, display_event(div, attributes=point_attributes))

p.js_on_event(events.MouseWheel, display_event(div, attributes=wheel_attributes))
p.js_on_event(events.Pan,        display_event(div, attributes=pan_attributes))
p.js_on_event(events.Pinch,      display_event(div, attributes=pinch_attributes))

output_file("js_events.html", title="JS Events Example")
show(layout)

Loading BokehJS ...

In [92]:
output_file("test.html")
save(p)

'C:\\Users\\chuvakin\\Desktop\\ScientificSerach\\test.html'